In [1]:
! pip install torch==1.12.0+cu111 torchvision==0.9.2+cu111 torchaudio===0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html -q
! pip install datasets -q
! pip install rouge -q
! pip install transformers[torch] -q

ERROR: Could not find a version that satisfies the requirement torch==1.12.0+cu111 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1)
ERROR: No matching distribution found for torch==1.12.0+cu111
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.0 MB/s eta 0:00:00


In [2]:
from datasets import Dataset, DatasetDict,load_dataset, load_from_disk

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/project2/df_train.csv')
test = pd.read_csv('/content/drive/MyDrive/project2/df_test.csv')
dev = pd.read_csv('/content/drive/MyDrive/project2/df_dev.csv')

In [5]:
train = train.sample(1600)
test = test.sample(400)
dev = dev.sample(400)


In [6]:
ds = DatasetDict()

ds['train'] = Dataset.from_pandas(train)
ds['test'] = Dataset.from_pandas(test)
ds['dev'] = Dataset.from_pandas(dev)

print(ds)

DatasetDict({
    train: Dataset({
        features: ['article', 'summary', '__index_level_0__'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['article', 'summary', '__index_level_0__'],
        num_rows: 400
    })
    dev: Dataset({
        features: ['article', 'summary', '__index_level_0__'],
        num_rows: 400
    })
})


In [7]:
from transformers import EncoderDecoderModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained("cahya/bert2bert-indonesian-summarization")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

In [8]:
tokenizer.bos_token, tokenizer.eos_token, tokenizer.get_vocab()[tokenizer.bos_token], tokenizer.get_vocab()[tokenizer.eos_token]

('[CLS]', '[SEP]', 3, 1)

In [9]:
prefix = tokenizer.bos_token + " "
posfix = " " + tokenizer.eos_token

def preprocess_function(examples):
    inputs = [prefix + str(doc) + posfix for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(text=examples["summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tokenized_ds = ds.map(preprocess_function, batched=True)
tokenized_ds

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'summary', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['article', 'summary', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 400
    })
    dev: Dataset({
        features: ['article', 'summary', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 400
    })
})

In [11]:
from transformers import BertConfig, EncoderDecoderConfig, EncoderDecoderModel

config_encoder = BertConfig(
    vocab_size = len(tokenizer.get_vocab()), # sesuaikan dengan vocab tokenizer
    hidden_size = 36,
    num_hidden_layers = 6,
    num_attention_heads = 6,
    intermediate_size = 768,
)
config_decoder = BertConfig(
    vocab_size = len(tokenizer.get_vocab()), # sesuaikan dengan vocab tokenizer
    hidden_size = 36,
    num_hidden_layers = 6,
    num_attention_heads = 6,
    intermediate_size = 768,
)

config = EncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)
model = EncoderDecoderModel(config=config)
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id

In [12]:
from transformers import DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [13]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='t5-small')

In [14]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/project2/results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    fp16=True
)

In [16]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [17]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:619: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:639: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Epoch,Training Loss,Validation Loss
1,No log,10.052692
2,10.178700,9.797307
3,9.804300,9.594294
4,9.531700,9.430651
5,9.321600,9.299652
6,9.321600,9.198112
7,9.164600,9.122004
8,9.053800,9.068943
9,8.981300,9.037578
10,8.944900,9.027087


Removed shared tensor {'decoder.cls.predictions.decoder.weight', 'decoder.cls.predictions.decoder.bias'} while saving. This should be OK, but check by verifying that you don't receive any warning while reloading
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:619: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:639: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions a

TrainOutput(global_step=4000, training_loss=9.372620361328124, metrics={'train_runtime': 448.9037, 'train_samples_per_second': 35.642, 'train_steps_per_second': 8.911, 'total_flos': 30296181350400.0, 'train_loss': 9.372620361328124, 'epoch': 10.0})

In [18]:
article = """
    TEMPO.CO, Jakarta - Menteri Badan Usaha Milik Negeri (BUMN) Erick Thohir membawa satu kasus baru ke Kejaksaan Agung RI sebagai upaya bersih-bersih perusahaan pelat merah.Tadi di dalam pembicaraan kami yang pertama adalah ada satu case. Satu kasus yang rencananya akan diserahkan kepada kami dan kasus ini memang cukup menarik, kata Jaksa Agung Sanitiar Burhanuddin di gedung Kejaksaan Agung, Jakarta Selatan, Senin, 6 Maret 2023.Namun Burhanuddin mengatakan belum bisa mengungkapkan kasus baru yang dimaksud. Ia menuturkan Kejaksaan Agung perlu mendalami terlebih dahulu ihwal kasus tersebut.Erick Thohir mengatakan membenarkan Kementerian BUMN menyerahkan temuan-temuan untuk ditindaklanjuti oleh Kejaksaan Agung. Namun ia sepakat dengan Jaksa Agung agar tidak bicara perihal kasus tersebut.Karena harus ada pendalaman dulu baru kita bicara. Mungkin kasih waktu 1-2 minggu, mungkin Pak Jampidsus, mungkin Pak Wamen Tiko menyampaikan kalau sudah dapat laporan tertulis dan detailnya, kata Erick Thohir.Menteri BUMN Erick Thohir bersama Wakil Menteri BUMN Kartiko Wirjoatmodjo menemui Jaksa Agung ST Burhanuddin dan Jaksa Agung Muda Tindak Pidana Khusus (Jampidsus) Febrie Adriansyah hari ini untuk membicarakan kasus-kasus korupsi yang melibatkan BUMN. Salah satunya ihwal pelimpahan pengelolaan aset PT Jiwasraya oleh Kejagung ke Kementerian BUMN.Pelimpahan aset JiwasrayaMenteri BUMN Erick Thohir mengatakan nilai aset Jiwasraya yang sudah diserahkan sebesar Rp 3,1 dalam bentuk surat berharga atau saham. Tahun ini masih ada Rp 1,4 triliun yang masih dalam proses penyerahan.Ini yang perlu kita sinkronisasikan supaya jangan penyelesaian dari Jiwasraya tertunda karena penyelesaian aset dari administrasi saja, kata Erick.Erick mengatakan masih ada kasus Jiwarsaya yang mesti diselesaikan, termasuk pengembalian aset-aset yang telah disita oleh Kejaksaan Agung. Enam bulan ke depan, ujar Erick, adalah momen terpenting penyelesaian Jiwasraya.Selanjutnya: aset yang diserahkan dalam bentuk sahamSementara itu, Kepala Pusat Penerangan Hukum Kejaksaan Agung I Ketut Sumedana mengatakan saat ini baru aset sitaan senilai Rp 3,1 triliun dari kasus Jiwasraya yang diserahkan kepada Kementerian BUMN. Aset ini dalam bentuk surat berharga atau saham.Bentuknya saham. Kalau dalam bentuk tanah belum karena belum ada yang dijual ya, kata Ketut.Sebelumnya, Majelis hakim Pengadilan Tindak Pidana Korupsi (Tipikor) Jakarta menjatuhkan vonis nihil terhadap Benny Tjokrosaputro selaku terdakwa perkara dugaan korupsi Asabri serta pencucian uang. Vonis nihil diberikan karena Benny sudah mendapat hukuman maksimal dalam kasus korupsi Jiwasraya."Karena terdakwa sudah dijatuhi pidana seumur hidup dalam perkara PT Asuransi Jiwasraya, maka pidana yang dijatuhkan dalam perkara a quo adalah pidana nihil," ujar ketua majelis hakim Ignatius Eko Purwanto di Pengadilan Tipikor Jakarta Pusat, Kamis 12 Januari 2023.Dalam kasus Asabri, jaksa menuntut Benny Tjokrosaputro dengan hukuman mati. Majelis hakim yang terdiri dari Ignatius Eko Purwanto, Saifuddin Zuhri, Teguh Santoso, Ali Mukhtarom dan Mulyono DWi Puryanto menetapkan bahwa Benny Tjokrosaputro terbukti melakukan korupsi dan pencucian uang, tapi majelis hakim tidak bisa menjatuhkan hukuman mati.Pada 26 Oktober 2020, Benny Tjokro divonis penjara seumur hidup oleh Majelis Hakim Pengadilan Tindak Pidana Korupsi Jakarta. Selain itu, majelis hakim mewajibkan Direktur PT Hanson International Tbk itu membayar yang pengganti sebanyak Rp 6 triliun.Hakim menyatakan Benny terbukti melakukan berbagai perbuatan yang membuat negara rugi Rp 16 triliun. Perbuatan itu di antaranya, pengelolaan saham dan reksa dana Jiwasraya, tanpa analisis dan tak sesuai prosedur. Selain korupsi, hakim menyatakan Benny terbukti melakukan pencucian uang.Perbuatan itu dilakukan bersama-sama dengan eks Direktur Utama Jiwasraya Hendrisman Rahim; Direktur Keuangan Jiwasraya 2013-2018, Hary Prasetyo; Kepala Divisi Investasi dan Keuangan Jiwasraya 2008-2014, Syahmirwan; dan Direktur PT Maxima Integra, Joko Hartomo Tirto. Keempat orang tersebut sudah lebih dulu divonis penjara seumur hidup.M ROSSENO AJI | EKA YUDHA SAPUTRA | CAESAR AKBARPilihan Editor:Jokowi Ungkit Kasus Asabri hingga Jiwasraya: Rakyat Nangis, Hanya Minta Duit Balik
"""
summary = """
    Menteri BUMN Erick Thohir dan Wakil Menteri BUMN Kartiko Wirjoatmodjo menyerahkan satu kasus baru ke Kejaksaan Agung untuk membersihkan perusahaan BUMN. Meski demikian, Jaksa Agung Sanitiar Burhanuddin belum dapat mengungkapkan detail kasus tersebut. Mereka juga membahas pelimpahan pengelolaan aset PT Jiwasraya oleh Kejagung ke Kementerian BUMN. Hingga saat ini, hanya aset senilai Rp 3,1 triliun dalam bentuk saham yang diserahkan. Ada pula kasus Jiwarsaya yang harus diselesaikan dalam enam bulan ke depan. Benny Tjokrosaputro, eks Direktur PT Hanson International Tbk divonis pidana seumur hidup karena terbukti melakukan korupsi dan pencucian uang dengan membuat negara rugi Rp 16 triliun.
"""

In [19]:
def predict_summary(document):
  device = model.device
  tokenized = tokenizer([document], max_length=512, truncation =True, padding ='longest',return_tensors='pt')
  tokenized = {k: v.to(device) for k, v in tokenized.items()}
  tokenized_result = model.generate(**tokenized, max_length=128)
  tokenized_result = tokenized_result.to('cpu')
  predicted_summary = tokenizer.decode(tokenized_result[0])
  return predicted_summary

In [20]:
predicted_summary = predict_summary(article)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


In [21]:
predicted_summary

'[CLS] [CLS]. [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP]. [SEP]. [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP]. [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP]. [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP]. [SEP] [SEP] [SEP] [SEP] [SEP] [SEP]. [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP]. [SEP] [SEP] [SEP] [SEP].'

In [22]:
summary

'\n    Menteri BUMN Erick Thohir dan Wakil Menteri BUMN Kartiko Wirjoatmodjo menyerahkan satu kasus baru ke Kejaksaan Agung untuk membersihkan perusahaan BUMN. Meski demikian, Jaksa Agung Sanitiar Burhanuddin belum dapat mengungkapkan detail kasus tersebut. Mereka juga membahas pelimpahan pengelolaan aset PT Jiwasraya oleh Kejagung ke Kementerian BUMN. Hingga saat ini, hanya aset senilai Rp 3,1 triliun dalam bentuk saham yang diserahkan. Ada pula kasus Jiwarsaya yang harus diselesaikan dalam enam bulan ke depan. Benny Tjokrosaputro, eks Direktur PT Hanson International Tbk divonis pidana seumur hidup karena terbukti melakukan korupsi dan pencucian uang dengan membuat negara rugi Rp 16 triliun.\n'

In [23]:
from rouge import Rouge

def get_rouge_scores(actual_summary, predicted_summary):
    rouge = Rouge()
    scores = rouge.get_scores(predicted_summary, actual_summary)
    return [scores[0]['rouge-1']['f'], scores[0]['rouge-2']['f'], scores[0]['rouge-l']['f']]

In [25]:
# rouge score of validation data

from tqdm import tqdm
import pandas as pd

rouge1_scores = []
rouge2_scores = []
rougel_scores = []

pred_summary_list = []

for i in tqdm(range(len(ds['test']))):

  doc = ds['test']['article'][i]
  pred_summary = predict_summary(doc)
  human_summary = ds['test']['summary'][i]

  score = get_rouge_scores(human_summary, pred_summary)

  rouge1_scores.append(score[0])
  rouge2_scores.append(score[1])
  rougel_scores.append(score[2])

  pred_summary_list.append(pred_summary)

eval = pd.DataFrame({
    "pred_summary" : pred_summary_list,
    "rouge1" : rouge1_scores,
    "rouge2" : rouge2_scores,
    "rougel" : rougel_scores,
})


100%|██████████| 400/400 [09:38<00:00,  1.45s/it]


In [26]:
# average rouge 1
eval['rouge1'].mean()

0.0

In [27]:
# average rouge 2
eval['rouge2'].mean()

0.0

In [28]:
# average rouge l
eval['rougel'].mean()

0.0

In [29]:
eval

,pred_summary,rouge1,rouge2,rougel
0,[CLS] [CLS]. [SEP] [SEP] [SEP] [SEP] [SEP] [SE...,0.0,0.0,0.0
1,[CLS] [CLS]. [SEP] [SEP] [SEP] [SEP] [SEP] [SE...,0.0,0.0,0.0
2,[CLS] [CLS]. [SEP] [SEP] [SEP] [SEP] [SEP] [SE...,0.0,0.0,0.0
3,[CLS] [CLS]. [SEP] [SEP] [SEP] [SEP] [SEP] [SE...,0.0,0.0,0.0
4,[CLS] [CLS]. [SEP] [SEP] [SEP] [SEP] [SEP] [SE...,0.0,0.0,0.0
...,...,...,...,...
395,[CLS] [CLS]. [SEP] [SEP] [SEP] [SEP] [SEP] [SE...,0.0,0.0,0.0
396,[CLS] [CLS]. [SEP] [SEP] [SEP] [SEP] [SEP] [SE...,0.0,0.0,0.0
397,[CLS] [CLS]. [SEP] [SEP] [SEP] [SEP] [SEP] [SE...,0.0,0.0,0.0
398,[CLS] [CLS]. [SEP] [SEP] [SEP] [SEP] [SEP] [SE...,0.0,0.0,0.0
